In [ ]:
import pandas as pd
import os
import graphviz

In [ ]:
# Define the directory containing data
directory = 'bike store data'

In [ ]:
# Get a list of all csv files in the directory
csv_files = {file for file in os.listdir(directory) if file.endswith('.csv')}

In [ ]:
# Function to find the potential primary keys for a dataframe
def find_potential_primary_keys(df):
    potential_primary_keys = []
    for column in df.columns:
        if df[column].nunique() == len(df):
            potential_primary_keys.append(column)
    return potential_primary_keys

In [ ]:
# Dictionary to store the DataFrames
dataframes = {}

# Dictionary to store the potential primary keys
primary_keys = {}

In [ ]:
# Read and process each CSV files
for file in csv_files:
    file_path = os.path.join(directory, file)
    df = pd.read_csv(file_path)
    dataframes[file] = df
    primary_keys[file] = find_potential_primary_keys(df)

    # Print the potential primary keys for the current file
    print(f"File: {file}")
    print(f"Potential Primary keys: {primary_keys[file]}")
    print("\n")

In [ ]:
# Dictionary to store foreign keys
foreign_keys = {}

In [ ]:
# Check if primary keys are foreign keys in other tables
for file, keys in primary_keys.items():
    for key in keys:
        for other_file, other_df in dataframes.items():
            if file != other_file and key in other_df.columns:
                foreign_keys[key] = []
                foreign_keys[key].append(other_file)

In [ ]:
# Dictionary to store table types
table_types = {}

In [ ]:
# Determine table types
for file in csv_files:
    is_fact = False
    for key in primary_keys[file]:
        if key in foreign_keys:
            is_fact = True
            break
    if is_fact:
        table_types[file] = 'Fact'
    else:
        table_types[file] = 'Dimension'

In [ ]:
# Print table types
for file, table_type in table_types.items():
    print(f"Table: {file}")
    print(f"Type: {table_type}")
    print("\n")

In [ ]:
# Create a graph to represent relationships using Graphviz
dot = graphviz.Digraph(comment = 'ER Diagram')

In [ ]:
# Add nodes for each table
for file in csv_files:
    label = f"{file}\n({table_types[file]})"
    dot.node(file, label = label, shape='box', style = 'filled', color = 'lightgray')

In [ ]:
# Add edges for primary key to foreign key relationships
for key, tables in foreign_keys.items():
    for table in tables:
        for file in primary_keys:
            if key in primary_keys[file]:
                dot.edge(file, table, label = key)

In [ ]:
output_path = 'er_diagram'
dot.render(output_path, view=True)
